In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import PrettyPrinter
pp = PrettyPrinter()

Calling real-time forex rates using live endpoint

In [2]:
url = "https://marketdata.tradermade.com/api/v1/live"

currency = "USDJPY,GBPUSD,UK100"
api_key = "q_csXrJmlzDRc-8kkAkN"
querystring = {"currency":currency,"api_key":api_key}
response = requests.get(url, params=querystring)
pp.pprint(response.json())

{'endpoint': 'live',
 'quotes': [{'ask': 155.883,
             'base_currency': 'USD',
             'bid': 155.872,
             'mid': 155.8775,
             'quote_currency': 'JPY'},
            {'ask': 1.25341,
             'base_currency': 'GBP',
             'bid': 1.25337,
             'mid': 1.25339,
             'quote_currency': 'USD'},
            {'ask': 8130.2,
             'bid': 8122.2,
             'instrument': 'UK100',
             'mid': 8126.2}],
 'requested_time': 'Thu, 02 May 2024 04:32:07 GMT',
 'timestamp': 1714624327}


In [3]:
df = pd.DataFrame(response.json()["quotes"])
df

,ask,base_currency,bid,mid,quote_currency,instrument
0,155.88300,USD,155.87200,155.87750,JPY,NaN
1,1.25341,GBP,1.25337,1.25339,USD,NaN
2,8130.20000,NaN,8122.20000,8126.20000,NaN,UK100


In [21]:
df = pd.DataFrame.from_dict(response.json()["quotes"], orient='columns', dtype=None, columns=None)
df

,ask,base_currency,bid,mid,quote_currency,instrument
0,157.49700,USD,157.49500,157.49600,JPY,NaN
1,1.24884,GBP,1.24882,1.24883,USD,NaN
2,8140.50000,NaN,8139.30000,8139.90000,NaN,UK100


In [4]:
df["instrument"] = np.where(df["base_currency"].isnull(),df["instrument"],df["base_currency"]+df["quote_currency"])
#df["instrument"] = df["base_currency"]+df["quote_currency"]
df["timestamp"] = response.json()["timestamp"]
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
df

,ask,base_currency,bid,mid,quote_currency,instrument,timestamp
0,155.88300,USD,155.87200,155.87750,JPY,USDJPY,2024-05-02 04:32:07
1,1.25341,GBP,1.25337,1.25339,USD,GBPUSD,2024-05-02 04:32:07
2,8130.20000,NaN,8122.20000,8126.20000,NaN,UK100,2024-05-02 04:32:07


Historical Data Collection (Daily)



In [5]:
url = "https://marketdata.tradermade.com/api/v1/historical"
date = "2021-03-15"
currency = "EURUSD"
querystring = {"currency":currency,"date":date, "api_key":api_key}
response = requests.get(url, params=querystring)
pp.pprint(response.json())

{'date': '2021-03-15',
 'endpoint': 'historical',
 'quotes': [{'base_currency': 'EUR',
             'close': 1.19284,
             'high': 1.19677,
             'low': 1.19111,
             'open': 1.19502,
             'quote_currency': 'USD'}],
 'request_time': 'Thu, 02 May 2024 04:33:02 GMT'}


In [6]:
df = pd.DataFrame(response.json()["quotes"])
df

,base_currency,close,high,low,open,quote_currency
0,EUR,1.19284,1.19677,1.19111,1.19502,USD


In [7]:
df = pd.DataFrame.from_dict(response.json()["quotes"], orient='columns', dtype=None, columns=None)
df

,base_currency,close,high,low,open,quote_currency
0,EUR,1.19284,1.19677,1.19111,1.19502,USD


Minute and Hour Historical

In [8]:
fx = ["EURUSD", "USDJPY"]
dates = ["2021-03-15-13:00"]
array = []
url = "https://marketdata.tradermade.com/api/v1/minute_historical"
for i in fx:
    for date in dates:
        querystring = {"currency":i,"date_time":date, "api_key":api_key}
        response = requests.get(url, params=querystring)
        array.append(response.json())
df = pd.DataFrame(array)
df

,close,currency,date_time,endpoint,high,low,open,request_time
0,1.19241,EURUSD,2021-03-15-13:00,minute_historical,1.19256,1.19234,1.19248,"Thu, 02 May 2024 05:14:36 GMT"
1,109.16900,USDJPY,2021-03-15-13:00,minute_historical,109.17350,109.15700,109.16000,"Thu, 02 May 2024 05:14:36 GMT"


Time Series Forex Data

In [9]:
url = "https://marketdata.tradermade.com/api/v1/timeseries?"
currency="USDJPY"
start_date="2021-03-01"
end_date="2021-03-22"
format="split"
interval="daily"

df =pd.read_json('https://marketdata.tradermade.com/api/v1/timeseries?currency='
              +currency+'&api_key='+api_key+'&start_date='+start_date+'&end_date='
              +end_date+'&format='+format+'&interval='+interval)
df = pd.DataFrame(df.quotes['data'], columns=df.quotes['columns'])
df.tail()

,date,open,high,low,close
11,2021-03-16,109.15850,109.2885,108.7720,108.99050
12,2021-03-17,109.00250,109.3260,108.7430,108.85450
13,2021-03-18,108.85550,109.3045,108.6245,108.94350
14,2021-03-19,108.92950,109.1320,108.5945,108.61999
15,2021-03-22,108.62151,108.9580,108.6080,108.83650
